# Ordinary Differential Equations

Differential equations are systems whose variables change with time. Mathematically, differentials are functions that contain derivatives of itself. When these systems are integrated, they provide analytical  functions that are dependent with time. Integration of differentials can be performed by calculus, or numerically. 

INSERT image here 

Ordinary differential equations (ODES) are equations that have a single dependent variables. Partial differential equations are equations that are dependent on 2 or more variables. Furthermore, the order of a differential equation is characterized by the highest derivative of an independent variable. 

Biological cell growth is a first order differential that describes the exponential increase of biomass concentration  (X) accumulation over time (assuming $ \mu _g  $  is constant ). 


$$ \frac{dX}{dt} = \mu_g X  $$

The Navier-Stokes equations in contrast, are 2nd order partial differentials describing particle position of a fluid as a function of the derivatives of speed, sheer, time and, pressure. 

$$ \rho \big( \frac{du}{dt} +u \frac{du}{dx} + v \frac{dv}{dy} + w \frac{dw}{dz} \big) = - \frac{dP}{dx} + \mu \big( \frac{d^2u}{dx^2} + \frac{d^2v}{dy^2} +  \frac{d^2w}{dz^2}    \big)  $$ 

The scope of this tutorial focuses on  first order, ordinary differential equations. 



When ODE's are solved, some have an analytical solution. For example, solving the Biomass growth equation gives this analytical solution.

$$ X = X_0 e^{\mu _g (t-t_0)} $$

Where $ X_0 $  and $ t_0 $  are initial conditions of the cell concentration and lag time.  

If you have worked with numerical methods you may have come across solving differentials by using this form:

$$ Value_{new} = Value_{old} + slope*stepsize  $$ 


$$  \frac{dX}{dt} = \mu_g X$$

$$ \frac{X_{i+1}-X_i}{t_{i+1} - t{i}} = \mu_g X_i  $$

$$ X_{i+1}  =   X_{i}+ \mu _g X_i  \Delta t $$ 